# Autores
- Arturo Cristián Díaz López - A01709522
- José Emiliano Riosmena Castañón - A01704245
- Diego Vega Camacho - A0170442

# Descripción
El reto consiste en desarrollar un sistema multiagente para resolver una tarea cooperativa en un entorno 20x20 dinámicamente cambiante. El entorno del sistema multiagente es un mundo similar a una cuadrícula donde los agentes pueden moverse de su posición a una celda vecina si ya no hay ningún agente en esa ranura. En este entorno, la comida puede aparecer en cualquier celda menos en una. La celda especial, en la que no puede aparecer comida, se considera un depósito donde los agentes pueden traer y almacenar su comida. Un agente puede sólo puede saber si hay comida en una celda, si está visitándola. Inicialmente, la comida se coloca en algunas celdas aleatorias. Durante la ejecución, puede aparecer comida adicional dinámicamente en celdas seleccionadas al azar, excepto en la celda del depósito. Los agentes pueden tener/desempeñar diferentes roles (como explorador o recolector), comunicarse y cooperar para encontrar y recolectar alimentos de manera eficiente y efectiva.

# Puntos a considerar
- Inicialmente no hay comida en el entorno.
- La semilla para generación de números aleatorios será 12345.
- El depósito será generado al azar.
- Cada 5 segundos se colocará una unidad de comida en algunas celdas.
- La cantidad de celdas en las que colocará una unidad comida será definida al azar (entre 2 y 5 celdas).
- Se colocará un total de 47 unidades de comida.
- Número total de pasos (steps): 1500.
- La cantidad total de alimentos que se puede almacenar en el depósito es infinito.
- Hay un total de 5 agentes.
- Cuando una unidad de comida es encontrado por un explorador o por un agente que ya lleva la comida, la posición de la comida se marca y se comunica a otros agentes.
- Cuando un recolector encuentra una unidad comida, lo carga (gráficamente deberá cambia su forma para indicar que lleva comida). La capacidad máxima de comida que puede llevar un agentes es UNA unidad de comida.
- Inicialmente, los agentes no son informados sobre la posición del depósito, pero una vez que lo encuentran, todos saben dónde está.

# Criterios de evaluación
Los criterios que se utilizarán para evaluar sus soluciones y seleccionar a los tres primeros ganadores son los siguientes:
- Aplicación original, innovadora y efectiva de algoritmos computacionales para resolver problemas específicos.
- El rendimiento de la implementación. El rendimiento de la implementación ejecutable se medirá en función de la cantidad de alimentos que recolecte el sistema multiagente en una cantidad de pasos de simulación.
- La calidad de la descripción de análisis, diseño e implementación del sistema multiagente, la elegancia de su diseño e implementación.


In [1]:
# Importar las clases y funciones necesarias de Mesa
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# Importar las bibliotecas de visualización
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Configuraciones específicas para la animación
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Configurar la semilla para la generación de números aleatorios
import random
import numpy as np
import pandas as pd
import time
random.seed(12345)

In [ ]:
class Food(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.marked = False

In [ ]:
class Storage(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

In [26]:
class FoodExplorer(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = "food_explorer"
        self.depot_pos = None
        self.food_found = []

    def move_random(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def find_depot(self):
        if self.depot_pos is None:
            if self.model.grid.is_cell_empty(self.model.depot_pos):
                self.depot_pos = self.model.depot_pos
                print(f"Explorer {self.unique_id}: Depot found at {self.depot_pos}")
                self.model.communicate_depot_location()

    def communicate_depot_location(self):
        # Define this method in your model to handle communication of depot location.
        pass

    def mark_food(self):
        if self.pos not in self.food_found:
            self.food_found.append(self.pos)
            print(f"Explorer {self.unique_id}: Food found at {self.pos}")
            self.communicate_food_location()

    def communicate_food_location(self):
        # Define this method in your model to handle communication of food locations.
        pass

    def step(self):
        self.move_random()
        self.find_depot()


In [27]:
class food_collector(Agent):
    def __init__ (self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = "food_collector"
        self.depot_pos = None
        self.food_carried = 0

    def move_random(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def deposit_food(self):
        if self.food_carried > 0 and self.model.grid.is_cell_empty(self.depot_pos):
            self.model.grid.place_agent(self, self.depot_pos)
            self.model.food_deposited += self.food_carried
            print("Collector {}: Food deposited at {}".format(self.unique_id, self.depot_pos))
            self.food_carried = 0

    def collect_food(self):
        if self.pos in self.model.food_pos:
            food_agent = self.model.grid.get_cell_list_contents([self.pos])[0]
            self.food_carried += 1
            print("Collector {}: Food collected at {}. Total Food Carried {}".format(self.unique_id, self.pos, self.food_carried))
            food_agent.mark_collected()
            self.deposit_food()

    def find_depot(self):
        if self.depot_pos == None:
            if self.model.grid.is_cell_empty(self.model.depot_pos):
                self.depot_pos = self.model.depot_pos
                print("Collector {}: Depot found at {}".format(self.unique_id, self.depot_pos))
                # self.model.communicate_depot_location()

    def step(self):
        self.move_random()
        self.find_depot()
        self.collect_food()

In [28]:
"""
class food_model(Model):
    def __init__(self, width, height, num_agents=5, max_food=47):
        self.random.seed(12345)
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(width, height, True)
        self.running = True
        self.food_layers = np.zeros((width, height), dtype=np.int8)
        self.known_food = np.zeros((width, height), dtype=np.int8)
        self.storage_location = None
        self.food = max_food
        self.spawned_food = 0

        self.datacollector = DataCollector(
            model_reporters={"Food": lambda m: np.sum(m.food_layers),
                             "Known Food": lambda m: np.sum(m.known_food),
                             "Storage Location": lambda m: m.storage_location,
                             "Agents": lambda m: m.schedule.get_agent_count(),})
        
        # Crear instancias de agentes
        self.create_agents(3, food_collector, "food_collector_")
        self.create_agents(2, food_explorer, "food_explorer_")
        self.create_storage()

    def create_agents(self, num_agents, agent_class, agent_type):
        used_positions = set()

        for i in enumerate(num_agents):
            while True:
                x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
                while (x, y) in used_positions:

                    if (x, y) not in used_positions and self.grid.is_cell_empty((x, y)):
                        agent = agent_class((agent_type, i), self)
                        self.grid.place_agent(agent, (x, y))
                        self.schedule.add(agent)

                        used_positions.add((x, y))
                        break

    def create_food(self, max_food):
        cells = [(x, y) for x in range(self.grid.width) for y in range(self.grid.height) 
                 if self.grid.is_cell_empty((x, y)) and self.food_layers[x, y] == 0]
        
        num_food = min(self.random.randrange(2, 6), max_food - self.spawned_food, len(cells))

        for i in range(num_food):
            x, y = self.random.choice(cells)
            self.food_layers[x, y] = 1
            self.initial_food_layers[x, y] = 1
            self.spawned_food += 1
            cells.remove((x, y))

    def create_storage(self):
        while True:
            x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
            if self.grid.is_cell_empty((x, y)):
                self.storage_location = (x, y)
                break

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

        if self.schedule.steps % 5 == 0:
            self.create_food(self.num_food)
"""

In [ ]:
def getGrid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for (content, (x, y)) in model.grid.coord_iter():
        for con in content:
            if isinstance(con, Food):
                grid[x][y] = 1
            elif isinstance(con, Rappi):
                grid[x][y] = 2
            elif isinstance(con, Storage):
                grid[x][y] = 3
    
    return grid

In [ ]:
class Delivery(Model):
    def __init__(self, width, height, num_agents, num_food, delay):
        self.num_agents = num_agents
        self.num_food = num_food
        self.total_food = num_food
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True
        self.food_deposited = 0
        self.positions = np.zeros((width, height))
        self.food_id = 0
        self.current_spawn = time.time()
        self.storage_pos = None
        self.marked_food = []
        self.is_in_storage = False
        self.delay = delay
        self.agent_column = None
        self.datacollector = DataCollector(
            model_reporters={"Grid": getGrid})

        self.spawn_agents()
        self.spawn_storage()

    def spawn_agents(self):
        self.agent_column = self.grid.width // self.num_agents
        column = 0
        for i in range(self.num_agents):
            Rappi_deliver = "Rappi_" + str(i)
            while True:
                x_values = (column, column + self.agent_column - 1) 
                x, y = self.random.randrange(x_values), self.random.randrange(self.grid.height)
                if self.grid.is_cell_empty((x, y)):
                    agent = Rappi(Rappi_deliver, self, x_values)
                    self.grid.place_agent(agent, (x, y))
                    self.schedule.add(agent)
                    self.grid.place_agent(agent, (x, y))
                    break
                
            column += self.agent_column
    
    def spawn_storage(self):
        storage = "storage"
        x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
        agent = Storage(storage, self)
        self.schedule.add(agent)
        self.grid.place_agent(agent, (x, y))

    def spawn_food(self):
        food = random.randint(2, 5)

        if food > self.num_food:
            food = self.num_food

        for i in range(food):
            burger = "food_" + str(self.food_id)
            while True:
                x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
                if self.grid.is_cell_empty((x, y)):
                    agent = Food(burger, self)
                    self.schedule.add(agent)
                    self.grid.place_agent(agent, (x, y))
                    self.food_id += 1
                    break

        self.num_food -= food

    def finish(self):
        if self.food_deposited == self.total_food:
            self.running = False

    def step(self):
        start = time.time()
        current = time.time()

        if current - self.current_spawn >= self.delay:
            self.spawn_food()
            self.current_spawn = current
        self.datacollector.collect(self)
        self.schedule.step()

        end = time.time()
        elapsed = end - start

        if elapsed < self.delay:
            time.sleep(self.delay - elapsed)
        
        self.finish()

In [29]:
# Configuración del modelo
WIDTH = 20
HEIGHT = 20
NUM_AGENTS = 5
MAX_FOOD = 47
NUM_DEPOTS = 1

In [30]:
model = food_model(WIDTH, HEIGHT, NUM_AGENTS, MAX_FOOD)

for i in range(1500):
    model.step()

data = model.datacollector.get_model_vars_dataframe()

TypeError: 'int' object is not iterable